In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import sklearn.metrics 
from math import sqrt

In [ ]:
data=pd.read_csv("master_df.csv")

In [ ]:
data

In [ ]:
cell_name=list(data["cell_num"].unique())
print(cell_name)

In [ ]:
data.set_index("cell_num",inplace=True)

In [ ]:
group_data=data.groupby(["cell_num"])

In [ ]:
X=[]
Y=[]
for i in cell_name:
    if(i not in ['b3c7', 'b3c16', 'b3c37', 'b3c44']):
        temp_data=group_data.get_group(i).head(75)
        temp_data=temp_data.drop(["cycle_life"],axis=1)
        x1=[]
        for j in range(len(temp_data)):
            x1.append(list(temp_data.iloc[j]))
        X.append(x1)
        Y.append(int(data.loc[i,"cycle_life"].head(1)))
    
print(X[0][0:2])
print(Y[0:2])

In [ ]:
print(len(X),len(Y))

In [ ]:
X=np.array(X)
X.shape

In [ ]:

X=np.reshape(X,(len(X),75*7))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
# Scale x and y (two scale objects)
X = sc_x.fit_transform(X)
Y = np.array(Y)
Y = sc_y.fit_transform(Y.reshape(-1, 1))
X_train, X_test, Y_train, Y_test,= train_test_split(X,Y,test_size=0.3,random_state=0)

In [ ]:
print("X_train:",len(X_train))
print("X_test:",len(X_test))
print("Y_train:",len(Y_train))
print("Y_test:",len(Y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()

In [ ]:
# Number of trees in random forest
n_estimators = [5, 10, 25, 50]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [3, 5, 10, 15, 20]
# max_depth.append(None)
criterion=['squared_error', 'absolute_error', 'poisson']
bootstrap=[ True, False]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion':criterion,
               'bootstrap':bootstrap
               }

print(random_grid)

In [ ]:
flag=1
import warnings
from tqdm import tqdm
warnings.simplefilter(action='ignore')
def test():
        MAPE_min=100.0
        best_rs=0
        for mf in tqdm([2,3,5,7,10,75]):
                prm = {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth' :None , 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0,
                        'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 6, 'n_estimators':35, 'n_jobs': None, 'oob_score': False,  'verbose': 0, 'warm_start': False}
                rf = RandomForestRegressor(**prm,max_features=mf )
                rf_random = rf
                rf_random.fit(X_train, Y_train)
                pred = rf_random.predict(X_test)
                pred_ = sc_y.inverse_transform(pred.reshape(-1, 1))
                Y_test_ = sc_y.inverse_transform(Y_test.reshape(-1, 1))
                MAPE = sklearn.metrics.mean_absolute_percentage_error(Y_test_, pred)
                # print(MAPE)
                if(MAPE_min>MAPE):
                        MAPE_min=MAPE
                        best_rs=mf
        print(best_rs)
# if(not flag):
#         test()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter =10, cv = 5, verbose=2, random_state=42, n_jobs = 8)
# rf_random = GridSearchCV(rf ,random_grid,cv=4,scoring='accuracy', verbose=10, n_jobs=8)
   

In [ ]:
if(flag):
        prm = {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0,
                'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 35, 'n_jobs': None, 'oob_score': False, 'random_state': 6, 'verbose': 0, 'warm_start': False}
        rf = RandomForestRegressor(**prm)
        # rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter =10, cv = 5, verbose=2, random_state=42, n_jobs = 8)
        # rf_random = GridSearchCV(rf ,random_grid,cv=4,scoring='accuracy', verbose=10, n_jobs=8)
        rf_random=rf
        rf_random.fit(X_train,Y_train)

In [ ]:
if(flag):   
    print(rf_random.get_params())

In [ ]:
if(flag):
    pred=rf_random.predict(X_test)
    pred = sc_y.inverse_transform(pred.reshape(-1, 1))
    Y_test = sc_y.inverse_transform(Y_test.reshape(-1, 1))
    print(Y_test[0:2])
    print(pred[0:2])
    mean_absolute_percentage_error=sklearn.metrics.mean_absolute_percentage_error(Y_test,pred)
    mean_absolute_error=sklearn.metrics.mean_absolute_error(Y_test,pred)
    mean_sqaured_error=sklearn.metrics.mean_squared_error(Y_test,pred,squared=True)
    root_mean_sqaured_error=sklearn.metrics.mean_squared_error(Y_test,pred,squared=False)

In [ ]:
if(flag):
    print("MAPE:",mean_absolute_percentage_error*100,"%")
    print("MAE:",mean_absolute_error)
    print("MSE:",mean_sqaured_error)
    print("RMSE:",root_mean_sqaured_error)

In [ ]:
# print("y_actual y_pred diff")
# for i in range(len(Y_test)):
#     print(Y_test[i],pred[i],(abs(Y_test[i]-pred[i])/Y_test[i])*100)

In [ ]:
import matplotlib.pyplot as plt
def predvsactual():
    fig = plt.figure(figsize=(12, 5))
    markerline1, stemlines1, baseline1 = plt.stem(Y_test)
    markerline1.set_markerfacecolor('red')
    markerline, stemlines, baseline = plt.stem(pred)
    markerline.set_markerfacecolor('blue')
    plt.legend(['Actual_cycle_life', 'Predicted_cycle_life'])
    plt.xlabel('Cells')
    plt.ylabel('Cycle Life')
    plt.show()
    
predvsactual()
